In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
p=print
# 将上述list中每一项array拼接成一维的数组
arr_list = [np.array(3.1914115, dtype=np.float32), np.array(2.9425068, dtype=np.float32), np.array(3.060446, dtype=np.float32), np.array(3.191684, dtype=np.float32)]
concatenated_arr = np.vstack(arr_list)
l1=[[11,12,13],[21,22,23]]
l1=np.array(l1)
l2=np.array([[31,32,33],[41,42,43]])
print(l1,l1.shape)
print(l2,l2.shape)
# print(df(l))
# m=np.concatenate([l1,l2],0)
m=np.stack([l1,l2,l2],axis=1)
p(m,m.shape)
# arrays = [np.random.randn(3, 4) for _ in range(10)]
# arrays=np.array(arrays)
# print(arrays.shape)
# m=np.stack(arrays,axis=-1)
# p(m.shape)

In [ ]:
import math
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets,transforms
import torchvision
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
# 防止torch包与Anaconda环境中的同一个文件出现了冲突，画不出图
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
train_data=datasets.MNIST(root='./',train=True,download=True)
test_data=datasets.MNIST(root='./',train=False,download=True)

x_train=train_data.data
y_train=train_data.targets
print(x_train.shape,y_train.shape)


https://blog.csdn.net/weixin_55529678/article/details/128270833
https://steven-cloud.blog.csdn.net/article/details/90815082?spm=1001.2101.3001.6650.1&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-1-90815082-blog-121672906.235%5Ev29%5Epc_relevant_default_base3&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-1-90815082-blog-121672906.235%5Ev29%5Epc_relevant_default_base3&utm_relevant_index=2

In [1]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-
'''
@ author: 何雨轩
@ title: homework 2: MINST
@ description:预测图片分类
MNIST 数据集来自美国国家标准与技术研究所, National Institute of Standards and Technology (NIST).
使用卷积神经网络CNN，分类交叉熵Cross-Entropy做损失函数，量SGD做优化
器做图片的分类
net的output是分类的数目，先用classes储存所有类别，之后用
torch.max(pred,dim=1)[1]返回分类可能性最大的那个类的index

softmax()是多分类的logistic，用Cross-Entropy做损失函数
@ v0.1: 2023-04-15
自带dataset的函数：len(s)查看数据量,s.data查看x数据,s.targets查看y
回顾TensorDataset(x,y):把tensor打包为dataset
torch.FloatTensor(numpy)，把numpy转化为小数点形式的tensor


'''
import math
import os
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
# 防止torch包与Anaconda环境中的同一个文件出现了冲突，画不出图
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


def same_seed(seed):
    '''固定seed保证复现'''
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def split(x, y, valid_ratio):
    '''拆分为train和valid'''
    n_valid = int(len(x) * valid_ratio)
    n_train = len(x) - n_valid
    x_train, x_valid = random_split(x, [n_train, n_valid],
    torch.Generator().manual_seed(config.seed))
    y_train, y_valid = random_split(y, [n_train, n_valid],
    torch.Generator().manual_seed(config.seed))
    L=list(map(np.array,[x_train, y_train, x_valid, y_valid]))
    return map(torch.FloatTensor, L)


class MyModel(nn.Module):
    '''定义模型
    * input_size: 自变量维度'''

    def __init__(self):
        super(MyModel, self).__init__()
        self.conv = nn.Sequential(# (1,28,28)
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5, 5)),nn.ReLU(),# (6,24,24)
            nn.MaxPool2d((2,2)),# (6,12,12)
            nn.Conv2d(6, 12, 3),nn.ReLU(),# (12,10,10)
            nn.MaxPool2d((2,2))# (12,5,5)
        )

        self.fc = nn.Sequential(
            nn.Linear(12*5*5, 128),nn.BatchNorm1d(128), # 归一化
            nn.ReLU(),
            nn.Linear(128, 32), nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Linear(32, 10))

    def forward(self, x):
        print(x.shape)
        x = self.conv(x)
        print(x.shape)
        x = x.view(-1,12*5*5)
        x = self.fc(x)
        print(x.shape)
        exit(0)
        return x


def trainer(train_loader, valid_loader, model):
    #===prepare===
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=config.learning_rate)
    best_loss = math.inf
    record_train_loss = []
    record_valid_loss = []

    for epoch in range(config.n_epoches):
        #===train mode===
        model.train()
        loss_record = []
        train_loop = tqdm(train_loader, position=0, ncols=100, leave=False)
        for x, y in train_loop:
            x, y = x.to(config.device), y.to(config.device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            # 清零梯度，反向传播，更新权重
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # 进度条设置
            train_loop.set_description(f'Epoch [{epoch}/{config.n_epoches}]')
            train_loop.set_postfix({'loss': loss.item()})
            loss_record.append(loss.item())
        record_train_loss.append(np.mean(loss_record))

        #===evaluate mode===
        model.eval()
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(config.device), y.to(config.device)
            with torch.no_grad():  # 减少内存损耗
                pred = model(x)
                loss = criterion(pred, y)
            loss_record.append(loss.item())
        record_valid_loss.append(np.mean(loss_record))

        #===early stopping===
        if record_valid_loss[-1] < best_loss:
            best_loss = record_valid_loss[-1]
            print(f'Now model with loss {best_loss:.3f}... from epoch {epoch}')
            early_stop_count = 0
        else:
            early_stop_count += 1
        if early_stop_count >= config.early_stop:
            print(f'Model is not improving for {config.early_stop} epoches. The last epoch is {epoch}.')
            break
    torch.save(model.state_dict(), config.save_model(best_loss))
    print(f'Saving model with loss {best_loss:.3f}... from epoch {epoch}')
    return record_train_loss, record_valid_loss, best_loss


def loss_plot(train_loss, valid_loss):
    '''画损失图，训练误差和泛化误差'''
    plt.figure()
    plt.plot(train_loss, label='train_loss')
    plt.plot(valid_loss, label='valid_loss')
    plt.yscale('log')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('training loss')
    plt.legend()
    plt.savefig(config.save('training loss.png'))
    plt.show()


def predict(test_data, model):
    '''注意这里载入data不是loader一批批载入
    返回pred的值，错误率，错误的坐标'''
    model.eval()
    preds = []
    incorrect_index = []
    for i, x, y in tqdm(enumerate(test_data), ncols=50):
        x, y = x.to(config.device), y.to(config.device)
        with torch.no_grad():
            output = model(x)
            _, y_pred = torch.max(output, 1)
            if (y_pred != y):
                incorrect_index.append(i)
    return preds, len(incorrect_index) / len(test_data), incorrect_index


class config:
    '''超参数设定，用`print(pd.DataFrame([config.__dict__]))`查看当前参数'''

    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.seed = 45
        self.batch_size = 512
        self.valid_ratio = 0.1
        self.folder = 'run'
        #-==Important Hyperparameters===
        self.early_stop = 3000
        self.learning_rate = 1e-5
        self.n_epoches = 2

    def save(self, path: str):
        if not os.path.exists(self.folder):
            os.makedirs(self.folder)
        return os.path.join(self.folder, path)

    def save_model(self, loss):
        path = f'model_{loss:.3f}.ckpt'
        return self.save(path)


if __name__ == '__main__':
    config = config()
    same_seed(config.seed)
    # print(pd.DataFrame([config.__dict__]))
    # print(config.save('model.ckpt'))
    # exit(0)
    print(config.device)
    # 55000张训练集，5000张验证集
    # 训练集图片对应的标签
    # 10000张测试集
    # 测试集图片对应的标签

    #===data processing===
    # 将原数据<class 'PIL.Image.Image'>转成tensor
    train_data = datasets.MNIST(root='./', train=True, download=True,transform=transforms.ToTensor())
    test_data = datasets.MNIST(root='./', train=False, download=True,transform=transforms.ToTensor())

    x_train = train_data.data.numpy()
    y_train = train_data.targets.numpy()

    x_train, y_train, x_valid, y_valid = split(x_train, y_train,config.valid_ratio)

    for i in range(12):
        plt.subplot(3, 4, i + 1)
        plt.tight_layout()  # 自动调整子图参数
        plt.imshow(x_train[i], cmap='gray')
        plt.title('label:{}'.format(y_train[i]))
        plt.xticks([])
        plt.yticks([])
    # plt.show()

    train_dataset = TensorDataset(x_train, y_train)
    valid_dataset = TensorDataset(x_valid, y_valid)

    train_loader = DataLoader(train_data,
                              batch_size=config.batch_size,
                              shuffle=True,
                              pin_memory=True,
                              drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=config.batch_size,
                              shuffle=True,
                              pin_memory=True,
                              drop_last=True)

    #===training===
    model = MyModel().to(config.device)
    print(model)
    train_loss, valid_loss, best_loss = trainer(train_loader, valid_loader,model)

    #===plot loss===
    loss_plot(train_loss, valid_loss)

    #===predict===
    model = MyModel().to(config.device)
    # model.load_state_dict(torch.load(config.save('model.ckpt')))
    model.load_state_dict(torch.load(config.save_model(best_loss)))
    y_test = predict(test_data, model)
    df = pd.DataFrame(y_test, columns=['test_predict'])
    # df.to_csv('test_predict.csv')


cpu
MyModel(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=300, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=32, bias=True)
    (4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=10, bias=True)
  )
)


  0%|                                                                       | 0/117 [00:00<?, ?it/s]

torch.Size([512, 1, 28, 28])
torch.Size([512, 12, 5, 5])
torch.Size([512, 10])


Epoch [0/2]:   1%|▎                                      | 1/117 [00:01<02:46,  1.44s/it, loss=2.38]

torch.Size([512, 1, 28, 28])
torch.Size([512, 12, 5, 5])
torch.Size([512, 10])


...

torch.Size([512, 1, 28, 28])
torch.Size([512, 12, 5, 5])
torch.Size([512, 10])


Epoch [0/2]:  53%|████████████████████▏                 | 62/117 [01:34<01:06,  1.21s/it, loss=2.38]

torch.Size([512, 1, 28, 28])
torch.Size([512, 12, 5, 5])
torch.Size([512, 10])


Epoch [0/2]:  54%|████████████████████▍                 | 63/117 [01:35<01:03,  1.18s/it, loss=2.38]

torch.Size([512, 1, 28, 28])
torch.Size([512, 12, 5, 5])
torch.Size([512, 10])


https://zhuanlan.zhihu.com/p/77471991

In [ ]:
def trainer(train_loader, valid_loader, model):
    #===prepare===
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=config.learning_rate)
    best_loss = math.inf
    record_train_loss = []
    record_valid_loss = []

    for epoch in range(config.n_epoches):
        #===train mode===
        model.train()
        loss_record = []
        train_loop = tqdm(train_loader, position=0, ncols=100, leave=False)
        for i, (x, y) in enumerate(train_loop):
            print(i,x,y)
            exit(0)
            x, y = x.to(config.device), y.to(config.device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            # 清零梯度，反向传播，更新权重
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # 进度条设置
            train_loop.set_description(f'Epoch [{epoch}/{config.n_epoches}]')
            train_loop.set_postfix({'loss': loss.item()})
            loss_record.append(loss.item())
        record_train_loss.append(np.mean(loss_record))

        #===evaluate mode===
        model.eval()
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(config.device), y.to(config.device)
            with torch.no_grad():  # 减少内存损耗
                pred = model(x)
                loss = criterion(pred, y)
            loss_record.append(loss.item())
        record_valid_loss.append(np.mean(loss_record))

        #===early stopping===
        if record_valid_loss[-1] < best_loss:
            best_loss = record_valid_loss[-1]
            print(f'Now model with loss {best_loss:.3f}... from epoch {epoch}')
            early_stop_count = 0
        else:
            early_stop_count += 1
        if early_stop_count >= config.early_stop:
            print(f'Model is not improving for {config.early_stop} epoches. The last epoch is {epoch}.')
            break
    torch.save(model.state_dict(), config.save_model(best_loss))
    print(f'Saving model with loss {best_loss:.3f}... from epoch {epoch}')
    return record_train_loss, record_valid_loss, best_loss
train_loss, valid_loss, best_loss = trainer(train_loader, valid_loader,model)

In [ ]:
https://blog.csdn.net/AugustMe/article/details/94854890